In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

import tkinter as tk
from tkinter import messagebox

In [ ]:
# Load your dataset
df = pd.read_csv("/content/Text Summarization Data.csv" )


In [ ]:
# Display column names and number of features
print("Columns in the dataset:")
print(df.columns)

print("\nNumber of features (columns):", len(df.columns))

Columns in the dataset:
Index(['id', 'article', 'highlights'], dtype='object')

Number of features (columns): 3


In [ ]:
# We only need the 'article' and 'highlights' columns
df = df[['article', 'highlights']]

In [ ]:
# Drop rows with missing data
df.dropna(inplace=True)

In [ ]:
# Reset index after dropping rows
df.reset_index(drop=True, inplace=True)

In [ ]:
df.shape

(13368, 2)

In [ ]:
df = df.rename(columns={'article': 'text', 'highlights': 'summary'})

In [ ]:
df.head()

,text,summary
0,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...


In [ ]:
# Tokenizer and Model setup
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Freeze encoder layers for faster training
for param in model.encoder.parameters():
    param.requires_grad = False

In [ ]:
# Tokenization function
def tokenize_data(df, max_input_length=256, max_target_length=64):
    inputs = tokenizer(df['text'].tolist(), padding=True, truncation=True, max_length=max_input_length, return_tensors='pt')
    targets = tokenizer(df['summary'].tolist(), padding=True, truncation=True, max_length=max_target_length, return_tensors='pt')
    return inputs, targets

In [ ]:
# Split dataset
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
train_inputs, train_targets = tokenize_data(train_df)
val_inputs, val_targets = tokenize_data(val_df)

In [ ]:
# Custom dataset class for PyTorch DataLoader
class SummarizationDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'labels': self.targets['input_ids'][idx]
        }

In [ ]:
# Create DataLoader
train_dataset = SummarizationDataset(train_inputs, train_targets)
val_dataset = SummarizationDataset(val_inputs, val_targets)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4)


In [ ]:
# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,  # Use mixed precision
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,2.096000,1.868806
2,2.046000,1.849761
3,1.991900,1.845737
4,1.954600,1.840342
5,1.970000,1.834610
6,1.967600,1.830175
7,1.925500,1.833566
8,1.922300,1.833606
9,1.894800,1.833100
10,1.903400,1.831772


TrainOutput(global_step=30080, training_loss=1.970450898941527, metrics={'train_runtime': 2346.9247, 'train_samples_per_second': 51.263, 'train_steps_per_second': 12.817, 'total_flos': 8141486067548160.0, 'train_loss': 1.970450898941527, 'epoch': 10.0})

In [ ]:
model.save_pretrained('./fine_tuned_t5')
tokenizer.save_pretrained('./fine_tuned_t5')

('./fine_tuned_t5/tokenizer_config.json',
 './fine_tuned_t5/special_tokens_map.json',
 './fine_tuned_t5/spiece.model',
 './fine_tuned_t5/added_tokens.json')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('./fine_tuned_t5')
tokenizer = T5Tokenizer.from_pretrained('./fine_tuned_t5')


In [ ]:

# Function to summarize text
def summarize_text(text, max_length=150, num_beams=4):
    # Preprocess the input text
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the summary with beam search
    summary_ids = model.generate(inputs, max_length=max_length, num_beams=num_beams, early_stopping=True)

    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Example input text (you can replace it with your own)
input_text = """
Machine learning is a branch of artificial intelligence that focuses on building systems that can learn from and make decisions based on data.
It involves the use of algorithms and statistical models to analyze patterns in data and make predictions or decisions without being explicitly programmed.
Machine learning has become a significant area of research and application across various industries, from healthcare to finance, enabling systems to improve their performance over time through experience.
The primary goal of machine learning is to develop algorithms that can generalize from previous observations, allowing them to perform well on unseen data.
"""

# Get the summary
summary = summarize_text(input_text)

print("Original Text:")
print(input_text)
print("\nSummary:")
print(summary)


Original Text:

Machine learning is a branch of artificial intelligence that focuses on building systems that can learn from and make decisions based on data.
It involves the use of algorithms and statistical models to analyze patterns in data and make predictions or decisions without being explicitly programmed.
Machine learning has become a significant area of research and application across various industries, from healthcare to finance, enabling systems to improve their performance over time through experience.
The primary goal of machine learning is to develop algorithms that can generalize from previous observations, allowing them to perform well on unseen data.


Summary:
Machine learning is a branch of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves the use of algorithms and statistical models to analyze patterns in data and make predictions or decisions without being explicitly programmed.


In [ ]:
model.save_pretrained('./fine_tuned_t5')
tokenizer.save_pretrained('./fine_tuned_t5')


('./fine_tuned_t5/tokenizer_config.json',
 './fine_tuned_t5/special_tokens_map.json',
 './fine_tuned_t5/spiece.model',
 './fine_tuned_t5/added_tokens.json')

In [ ]:
!zip -r fine_tuned_t5.zip fine_tuned_t5


  adding: fine_tuned_t5/ (stored 0%)
  adding: fine_tuned_t5/spiece.model (deflated 48%)
  adding: fine_tuned_t5/added_tokens.json (deflated 83%)
  adding: fine_tuned_t5/model.safetensors (deflated 34%)
  adding: fine_tuned_t5/generation_config.json (deflated 29%)
  adding: fine_tuned_t5/special_tokens_map.json (deflated 85%)
  adding: fine_tuned_t5/config.json (deflated 62%)
  adding: fine_tuned_t5/tokenizer_config.json (deflated 94%)


In [ ]:
from google.colab import files
files.download('fine_tuned_t5.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>